## 원하는 책에 대한 리뷰만을 가져와서 json에 저장하는 코드.

### 필요한 파이썬 라이브러리

In [12]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv
from bson import ObjectId
import json
import re

### DB와 연결

In [13]:
load_dotenv()

connection_URI = os.getenv("MONGO_URI")
if not connection_URI:
    raise ValueError("환경 변수 'MONGO_URI'가 설정되어 있지 않습니다.")

try:
    client = MongoClient(connection_URI)
except Exception as e:
    raise ConnectionError(f"MongoDB 연결 실패: {e}")

db = client.get_database("crawling")

### DB 내에서 책 리뷰 추출

In [14]:
# 리뷰를 찾고자 하는 책 이름 (직접 수정)
book_name = "The Book Thief"

books_collection = db.get_collection("books")
reviews_collection = db.get_collection("reviews")

book = books_collection.find_one({"book_name": book_name})
if book:
    book_id = book['_id']
    
    reviews = list(reviews_collection.find({"book_id": book_id}))
    print(f"해당 책의 리뷰 수:{len(reviews)}")

    review_texts = [review.get("review", "") for review in reviews]
    safe_book_name = re.sub(r'[^a-zA-Z0-9가-힣]', '_', book_name)

    output_file = f"{safe_book_name}.json"
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(review_texts, f, ensure_ascii=False, indent=4)
        print(f'"review" 필드가 {output_file} 파일에 저장되었습니다.')
    except Exception as e:
        print(f"파일 저장 실패: {e}")
else:
    print("해당 제목의 책을 찾지 못했습니다.")

해당 책의 리뷰 수:500
"review" 필드가 The_Book_Thief.json 파일에 저장되었습니다.
